### Importing needed packages:

In [1]:
from dolfinx import fem, mesh
from dolfinx.fem.petsc import NonlinearProblem
from dolfinx.nls.petsc import NewtonSolver
import utils
import model
from setup import *
from configs import *
import numpy as np
from mpi4py import MPI
import ufl
from tqdm import tqdm

Setting up initial conditions:

In [2]:
# initial conditions
V_m_n.vector.set(V_MIN)
w_n.vector.set(1 / (V_MAX - V_MIN) ** 2)
V_m_nn.vector.set(V_MIN)
w_nn.vector.set(1 / (V_MAX - V_MIN) ** 2)

Defining variational form of the equations:

In [3]:
# Define variational problem
F = (
    A_m * (C_m * (V_m - V_m_n) + dt * model.I_ion(V_m, w)) * phi * dx(0)
    + dt * ufl.inner(ufl.dot(model.sigma_i, ufl.grad(V_m + u)), ufl.grad(phi)) * dx(0)
    - dt * A_m * model.I_app() * phi * dx(0)
)
F += (
    ufl.inner(ufl.dot(model.sigma_i + model.sigma_e, ufl.grad(u)), ufl.grad(psi))
    * dx(0)
    + ufl.inner(ufl.dot(model.sigma_i, ufl.grad(V_m)), ufl.grad(psi)) * dx(0)
    + ufl.inner(ufl.dot(model.sigma_t, ufl.grad(u)), ufl.grad(psi)) * dx(1)
)

Defining solver and solving an equation:

In [7]:
# step 1
V_m_tilde = 2 * V_m_n - V_m_nn


# step 2
F_w = (
    (1 / dt * (3 / 2 * w - 2 * w_n + 1 / 2 * w_nn) + model.g(V_m_tilde, w))
    * theta
    * ufl.dx(0)
)
def solve_w():
    problem = NonlinearProblem(F_w, w)
    solver = NewtonSolver(MPI.COMM_WORLD, problem)
    solver.rtol = 1e-0
    solver.atol = 1e-0
    solver.max_it = 5
    log.set_log_level(log.LogLevel.INFO)
    solver.solve(w)
    return w

w = solve_w()

# step 3
pass

# step 4
w.x.array

2023-12-17 18:25:01.516 (  12.024s) [main            ]    SparsityPattern.cpp:384   INFO| Column ghost size increased from 0 to 0


array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0.])

2023-12-17 18:25:01.516 (  12.024s) [main            ]       NewtonSolver.cpp:36    INFO| Newton iteration 0: r (abs) = 0 (tol = 1) r (rel) = -nan(tol = 1)
2023-12-17 18:25:01.516 (  12.024s) [main            ]       NewtonSolver.cpp:255   INFO| Newton solver finished in 0 iterations and 0 linear solver iterations.


In [8]:
w = solve_w()

2023-12-17 18:25:03.252 (  13.760s) [main            ]    SparsityPattern.cpp:384   INFO| Column ghost size increased from 0 to 0
2023-12-17 18:25:03.252 (  13.761s) [main            ]       NewtonSolver.cpp:36    INFO| Newton iteration 0: r (abs) = 0 (tol = 1) r (rel) = -nan(tol = 1)
2023-12-17 18:25:03.252 (  13.761s) [main            ]       NewtonSolver.cpp:255   INFO| Newton solver finished in 0 iterations and 0 linear solver iterations.


In [9]:
problem = NonlinearProblem(F, v)
solver = NewtonSolver(MPI.COMM_WORLD, problem)
solver.rtol = 1e-0
solver.atol = 1e-0
solver.max_it = 5
log.set_log_level(log.LogLevel.INFO)
n, converged = solver.solve(v)

TypeError: compute_integration_domains(): incompatible function arguments. The following argument types are supported:
    1. (integral_type: dolfinx::fem::IntegralType, meshtags: dolfinx.cpp.mesh.MeshTags_int32) -> List[Tuple[int, List[int]]]

Invoked with: <IntegralType.cell: 0>, <dolfinx.cpp.mesh.MeshTags_float64 object at 0x7fb3da716030>

In [ ]:
v.x.array